In [1]:
# !pip install folktables

In [2]:
#info about Data
# https://arxiv.org/pdf/2108.04884.pdf

#ACSDataSource ,ACSEmployment, ACSPublicCoverage, ACSIncome

-> (using ACSEmployment) from the paper: predict whether an individual is employed, after filtering the ACS PUMS data
sample to only include individuals between the ages of 16 and 90  

In [1]:
# !pip install torchsummary

In [2]:
# !python --version

In [1]:
import pandas as pd
from collections import OrderedDict
from typing import OrderedDict
import numpy as np
import copy
import os
import torch
import torch.nn as nn
from sklearn import preprocessing
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import matplotlib.pyplot as plt
import torch.nn.functional as F
import tqdm
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, Subset, random_split
import torch.utils.data as data_utils
import pickle

In [2]:
# detail about the state: load_acs.py

In [3]:

from folktables import ACSDataSource, ACSEmployment,ACSIncome
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["AL"], download=True)

In [4]:
# features_np, label_np, group_np = ACSEmployment.df_to_numpy(acs_data)
features, label, group = ACSEmployment.df_to_pandas(acs_data)

In [5]:
features

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P
0,19.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0
1,18.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0
2,53.0,17.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0
3,28.0,19.0,5.0,16.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
4,25.0,12.0,5.0,16.0,1.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47772,18.0,16.0,5.0,2.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0
47773,15.0,11.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
47774,10.0,6.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
47775,4.0,1.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,0.0,1.0,1.0


In [6]:
label

,ESR
0,False
1,False
2,False
3,False
4,False
...,...
47772,True
47773,False
47774,False
47775,False


In [7]:
merged_df=pd.concat([features, label], axis=1)
merged_df.head()

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,19.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
1,18.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,False
2,53.0,17.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,False
3,28.0,19.0,5.0,16.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
4,25.0,12.0,5.0,16.0,1.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,False


In [8]:
len(merged_df)

47777

In [9]:
group.value_counts()

RAC1P
1        35551
2        10115
9          835
6          556
8          491
3          182
5           33
7           13
4            1
dtype: int64

In [10]:
merged_df["RAC1P"].value_counts()

# 1: White alone – 2: Black or African American alone

1.0    35551
2.0    10115
9.0      835
6.0      556
8.0      491
3.0      182
5.0       33
7.0       13
4.0        1
Name: RAC1P, dtype: int64

In [11]:
pd.unique(merged_df["RAC1P"])

array([1., 2., 8., 9., 6., 3., 5., 7., 4.])

# data loader-torch

In [12]:
class DeepNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 14 : input shape
        self.layer1 = nn.Linear(16, 512)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(512, 256)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(256, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.dropout1(x)
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x    

# training with Loaders

In [13]:
model = DeepNet()

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # You can adjust the learning rate

# Move the model to the appropriate device (CPU or GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DeepNet(
  (layer1): Linear(in_features=16, out_features=512, bias=True)
  (act1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (layer2): Linear(in_features=512, out_features=256, bias=True)
  (act2): ReLU()
  (layer3): Linear(in_features=256, out_features=60, bias=True)
  (act3): ReLU()
  (output): Linear(in_features=60, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [14]:
# name_of_model='3_WW_WM_BW.pth'

In [15]:
# torch.save(model.state_dict(), name_of_model)

# Testing

In [16]:
# total client 4
name_of_model="../temp/my_model/global_model.pt"
model = DeepNet()
model.load_state_dict(torch.load(name_of_model))

<All keys matched successfully>

In [17]:
class DeepNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 14 : input shape
        self.layer1 = nn.Linear(16, 512)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(512, 256)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(256, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.dropout1(x)
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x    

In [18]:
model = DeepNet()
# model = Deep_wide_Net()
model.load_state_dict(torch.load(name_of_model))
model.eval()

DeepNet(
  (layer1): Linear(in_features=16, out_features=512, bias=True)
  (act1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (layer2): Linear(in_features=512, out_features=256, bias=True)
  (act2): ReLU()
  (layer3): Linear(in_features=256, out_features=60, bias=True)
  (act3): ReLU()
  (output): Linear(in_features=60, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [19]:
data_source_test = ACSDataSource(survey_year='2017', horizon='1-Year', survey='person')
acs_data_test = data_source_test.get_data(states=["AL"], download=True)
features_test, label_test, group_test = ACSEmployment.df_to_pandas(acs_data_test)
print(features_test.head(),"\n\n Lable: \n ", label_test.head())

   AGEP  SCHL  MAR  RELP  DIS  ESP  CIT  MIG  MIL  ANC  NATIVITY  DEAR  DEYE  \
0  73.0  10.0  4.0   0.0  1.0  0.0  1.0  1.0  4.0  1.0       1.0   2.0   2.0   
1  31.0  21.0  5.0   0.0  2.0  0.0  1.0  1.0  4.0  2.0       1.0   2.0   2.0   
2  41.0  17.0  1.0   0.0  2.0  0.0  1.0  1.0  4.0  1.0       1.0   2.0   2.0   
3  48.0  16.0  1.0   1.0  2.0  0.0  1.0  1.0  4.0  1.0       1.0   2.0   2.0   
4  16.0  13.0  5.0   2.0  2.0  2.0  1.0  1.0  0.0  1.0       1.0   2.0   2.0   

   DREM  SEX  RAC1P  
0   1.0  1.0    2.0  
1   2.0  2.0    1.0  
2   2.0  1.0    1.0  
3   2.0  2.0    1.0  
4   2.0  1.0    1.0   

 Lable: 
       ESR
0  False
1   True
2   True
3  False
4  False


In [20]:
merged_df_test=pd.concat([features_test, label_test], axis=1)
merged_df_test

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,73.0,10.0,4.0,0.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,False
1,31.0,21.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,True
2,41.0,17.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,True
3,48.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
4,16.0,13.0,5.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47640,20.0,18.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47641,18.0,16.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,True
47642,25.0,17.0,1.0,16.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,9.0,False
47643,24.0,17.0,5.0,16.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False


In [21]:
# # filter the dataframe for each combination

#White Men
WM_test = merged_df_test.loc[(merged_df_test['SEX'] == 1) & (merged_df_test['RAC1P'] == 1)]
#Black Men
BM_test = merged_df_test.loc[(merged_df_test['SEX'] == 1) & (merged_df_test['RAC1P'] == 2)]
#white Women
WW_test = merged_df_test.loc[(merged_df_test['SEX'] == 2) & (merged_df_test['RAC1P'] == 1)]
#Black Women
BW_test = merged_df_test.loc[(merged_df_test['SEX'] == 2) & (merged_df_test['RAC1P'] == 2)]

In [22]:
WM_BM_BW_test= pd.concat([WM_test,BM_test,BW_test])

WW_BW_BM_test= pd.concat([WW_test,BW_test,BM_test])

BM_WM_WW_test= pd.concat([BM_test,WM_test,WW_test])

WW_WM_BW_test= pd.concat([WW_test,WM_test,BW_test])

In [23]:
WW_BW_BM_test

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
1,31.0,21.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,True
3,48.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
17,72.0,21.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,True
21,58.0,19.0,1.0,6.0,1.0,0.0,1.0,1.0,4.0,4.0,1.0,2.0,2.0,2.0,2.0,1.0,False
24,42.0,19.0,1.0,1.0,2.0,0.0,3.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47630,21.0,19.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,False
47631,54.0,21.0,3.0,16.0,1.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,False
47633,35.0,13.0,5.0,16.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,False
47636,20.0,19.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,False


In [24]:
merged_df_test.groupby(['SEX', 'RAC1P']).size()

SEX  RAC1P
1.0  1.0      16783
     2.0       4988
     3.0        110
     4.0          1
     5.0         28
     6.0        290
     7.0          8
     8.0        253
     9.0        395
2.0  1.0      18044
     2.0       5653
     3.0         97
     4.0          1
     5.0         19
     6.0        327
     7.0          7
     8.0        215
     9.0        426
dtype: int64

In [25]:
merged_df_test.groupby(['ESR']).size()

ESR
False    28430
True     19215
dtype: int64

In [26]:
merged_df_test

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,73.0,10.0,4.0,0.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,False
1,31.0,21.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,True
2,41.0,17.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,True
3,48.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
4,16.0,13.0,5.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47640,20.0,18.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47641,18.0,16.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,True
47642,25.0,17.0,1.0,16.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,9.0,False
47643,24.0,17.0,5.0,16.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False


In [27]:
X_test= WW_BW_BM_test[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
y_test= WW_BW_BM_test[['ESR']]


X_test1= WW_BW_BM_test[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM','SEX', 'RAC1P']]
y_test1= WW_BW_BM_test[['ESR']]

# X_test= merged_df_test[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
# y_test= merged_df_test[['ESR']]


X_test1 = preprocessing.normalize(X_test1)
encoder = LabelEncoder()
encoder.fit(y_test1)
y_test1 = encoder.transform(y_test1)

len(X_test1)



C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


28685

In [28]:
WW_BW_BM_test.groupby(['SEX', 'RAC1P']).size()

SEX  RAC1P
1.0  2.0       4988
2.0  1.0      18044
     2.0       5653
dtype: int64

In [29]:
WW_BW_BM_test.groupby(['ESR']).size()

ESR
False    18161
True     10524
dtype: int64

In [30]:
WW_BW_BM_test.describe()

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P
count,28685.000000,28685.000000,28685.000000,28685.000000,28685.000000,28685.000000,28685.000000,28685.000000,28685.000000,28685.000000,28685.000000,28685.000000,28685.000000,28685.000000,28685.000000,28685.00000
mean,43.434025,15.549869,2.913718,2.501447,1.796549,0.642949,1.064424,1.242147,3.238208,1.858881,1.014816,1.958306,1.961304,1.832003,1.826111,1.37096
std,23.899621,5.777482,1.788744,4.344732,0.402572,1.830964,0.452720,0.670723,1.521557,1.222105,0.120818,0.199893,0.192873,0.479296,0.379020,0.48307
min,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.00000
25%,23.000000,14.000000,1.000000,0.000000,2.000000,0.000000,1.000000,1.000000,4.000000,1.000000,1.000000,2.000000,2.000000,2.000000,2.000000,1.00000
50%,45.000000,16.000000,3.000000,1.000000,2.000000,0.000000,1.000000,1.000000,4.000000,1.000000,1.000000,2.000000,2.000000,2.000000,2.000000,1.00000
75%,63.000000,20.000000,5.000000,2.000000,2.000000,0.000000,1.000000,1.000000,4.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.00000
max,94.000000,24.000000,5.000000,17.000000,2.000000,8.000000,5.000000,3.000000,4.000000,4.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000


In [31]:
X_test= torch.tensor(X_test1, dtype=torch.float32)
y_test = torch.tensor(y_test1, dtype=torch.float32).reshape(-1, 1)
print(X_test)
print(y_test)

tensor([[0.8083, 0.5475, 0.1304,  ..., 0.0521, 0.0521, 0.0261],
        [0.9408, 0.3136, 0.0196,  ..., 0.0392, 0.0392, 0.0196],
        [0.9562, 0.2789, 0.0133,  ..., 0.0266, 0.0266, 0.0133],
        ...,
        [0.8429, 0.3131, 0.1204,  ..., 0.0482, 0.0241, 0.0482],
        [0.5979, 0.5680, 0.1495,  ..., 0.0598, 0.0299, 0.0598],
        [0.5661, 0.5661, 0.1573,  ..., 0.0629, 0.0315, 0.0629]])
tensor([[1.],
        [0.],
        [1.],
        ...,
        [0.],
        [0.],
        [0.]])


In [32]:
BATCH=64

test_data = data_utils.TensorDataset(X_test, y_test)

test_loader = data_utils.DataLoader(test_data, batch_size=BATCH, shuffle=True)

In [33]:
import torch
from sklearn.metrics import precision_score, recall_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    correct = 0
    total = 0
    loss = 0
    predicted_labels = []
    true_labels = []

    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)

        loss += criterion(outputs, targets)

        predicted = outputs > 0.5
        
        
        predicted_labels.extend(predicted.cpu().numpy())
#         print(outputs)
        true_labels.extend(targets.cpu().numpy())

        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    print("Correct: ", correct, "Total: ", total)

    average_loss = loss / len(test_loader)
    accuracy = correct / total

    print(f"Testing Accuracy: {accuracy:.2%}")
    print(f"Testing Loss:", average_loss)

    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)

    print(f"Precision: {precision:.2%}")
    print(f"Recall: {recall:.2%}")


Correct:  22024 Total:  28685
Testing Accuracy: 76.78%
Testing Loss: tensor(0.4693)
Precision: 64.27%
Recall: 82.66%


In [ ]:
# 1-: True  0-: Fasle
# encoder.inverse_transform(y_test)